<a href="https://colab.research.google.com/github/zeligism/ConGAN/blob/main/GAN_%2B_SimSiam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mount drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Header

## Imports

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import glob
import random
import datetime
import yaml
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data_utils
import torchvision
import torchvision.utils as vutils
import torchvision.transforms as transforms
import torch.utils.tensorboard as tensorboard

from PIL import Image, ImageDraw
from math import log2
from pprint import pformat
from collections import defaultdict

## Utility Functions

### Report Utils

In [3]:
def plot_lines(losses_dict, filename=None, title=""):
    """
    Plots the losses of the discriminator and the generator.

    Args:
        filename: The plot's filename. If None, plot won't be saved.
    """

    plt.figure(figsize=(10,5))
    plt.title(title)
    for label, losses in losses_dict.items():
        plt.plot(losses, label=label)
    plt.xlabel("t")
    plt.legend()
    
    if filename is not None:
        plt.savefig(filename)
    
    plt.show()
    plt.close()


def create_progress_animation(frames, filename):
    """
    Creates a video of the progress of the generator on a fixed latent vector.

    Args:
        filename: The animation's filename.
    """

    fig = plt.figure(figsize=(8,8))
    plt.axis("off")
    ims = [[plt.imshow(img.permute(1,2,0), animated=True)]
           for img in frames]
    ani = animation.ArtistAnimation(fig, ims, blit=True)
    
    ani.save(filename)

    plt.close()


def generate_grid(generator, latent):
    """
    Check generator's output on latent vectors and return it.

    Args:
        generator: The generator.
        latent: Latent vector from which an image grid will be generated.

    Returns:
        A grid of images generated by `generator` from `latent`.
    """

    with torch.no_grad():
        fake = generator(latent).detach()

    image_grid = vutils.make_grid(fake.cpu(), padding=2, normalize=True, range=(-1,1))

    return image_grid



# Classes

## PyTorch Modules

### DCGAN

In [4]:
class ConvBlock(nn.Module):
    """
    Default stride and padding half the size of features,
    e.g. if input is [in_channels, 64, 64], output will be [out_channels, 32, 32].
    """
    def __init__(self, in_channels, out_channels, kernel_size=4, stride=2, padding=1,
                 use_batchnorm=True, use_spectralnorm=False, activation=None):
        super().__init__()

        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size,
                              stride=stride, padding=padding, bias=False)
        if use_spectralnorm:
            self.conv = nn.utils.spectral_norm(self.conv)
        self.batchnorm = nn.BatchNorm2d(out_channels) if use_batchnorm else None
        self.activation = nn.LeakyReLU(0.2, inplace=True) if activation is None else activation()

    def forward(self, x):
        x = self.conv(x)
        if self.batchnorm:
            x = self.batchnorm(x)
        x = self.activation(x)
        return x


class ConvTBlock(nn.Module):
    """
    Default stride and padding double the size of features,
    e.g. if input is [in_channels, 32, 32], output will be [out_channels, 64, 64].
    """
    def __init__(self, in_channels, out_channels, kernel_size=4, stride=2, padding=1,
                 use_batchnorm=True, use_spectralnorm=False, activation=None):
        super().__init__()

        self.convT = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=kernel_size,
                                        stride=stride, padding=padding, bias=False)
        if use_spectralnorm:
            self.convT = nn.utils.spectral_norm(self.convT)
        self.batchnorm = nn.BatchNorm2d(out_channels) if use_batchnorm else None
        self.activation = nn.ReLU(inplace=True) if activation is None else activation()

    def forward(self, x):
        x = self.convT(x)
        if self.batchnorm:
            x = self.batchnorm(x)
        x = self.activation(x)
        return x


class DCGAN_Discriminator(nn.Module):
    def __init__(self,
                 num_latents=16,
                 num_features=64,
                 image_channels=3,
                 image_size=64,
                 feature_multiplier=2,
                 max_features=512,
                 gan_type="gan",
                 fully_convolutional=True,
                 activation=None,
                 use_batchnorm=True,
                 use_spectralnorm=False,
                 D_block=ConvBlock):
        super().__init__()

        using_grad_penalty = gan_type in ("gan-gp", "wgan-gp")
        output_sigmoid = gan_type in ("gan", "gan-gp")

        block_config = {
            "activation": activation,
            "use_batchnorm": use_batchnorm and not using_grad_penalty,
            "use_spectralnorm": use_spectralnorm,
        }

        # Calculate intermediate image sizes
        image_sizes = [image_size]
        while image_sizes[-1] > 5:
            image_sizes.append(image_sizes[-1] // 2)
        latent_kernel = image_sizes[-1]  # should be either 3, 4, or 5
        num_layers = len(image_sizes) - 1

        # Calculate feature sizes
        features = [min(max_features, round(num_features * feature_multiplier**i))
                    for i in range(num_layers)]

        # Input layer
        self.input_layer = D_block(image_channels, features[0], **block_config)

        # Intermediate layers
        self.main_layers = nn.Sequential(*[
            D_block(in_features, out_features, **block_config)
            for in_features, out_features in zip(features, features[1:])
        ])

        # Output layer (feature_size = 3, 4, or 5 -> 1)
        if fully_convolutional:
            self.output_layer = nn.Sequential(
                nn.Conv2d(features[-1], num_latents, latent_kernel, bias=False),
                nn.Flatten(),
            )
        else:
            self.output_layer = nn.Sequential(
                nn.Flatten(),
                nn.Linear(features[-1] * latent_kernel**2, num_latents, bias=False)
            )

        # Add sigmoid activation if using regular GAN loss
        self.output_activation = nn.Sigmoid() if output_sigmoid else None

    def forward(self, x):
        x = self.input_layer(x)
        x = self.main_layers(x)
        x = self.output_layer(x)
        if self.output_activation:
            x = self.output_activation(x)
        # Remove H and W dimensions, infer channels dim (remove if 1)
        x = x.view(x.size(0), -1).squeeze(1)
        return x


class DCGAN_Generator(nn.Module):
    def __init__(self,
                 num_latents=100,
                 num_features=64,
                 image_channels=3,
                 image_size=64,
                 feature_multiplier=2,
                 max_features=512,
                 gan_type="gan",
                 fully_convolutional=True,
                 activation=None,
                 use_batchnorm=True,
                 use_spectralnorm=False,
                 G_block=ConvTBlock):
        super().__init__()

        block_config = {
            "activation": activation,
            "use_batchnorm": use_batchnorm,
            "use_spectralnorm": use_spectralnorm
        }

        # Calculate intermediate image sizes
        image_sizes = [image_size]
        while image_sizes[-1] > 5:
            image_sizes.append(image_sizes[-1] // 2)
        latent_kernel = image_sizes[-1]  # should be either 3, 4, or 5
        num_layers = len(image_sizes) - 1

        # Calculate feature sizes
        features = [min(max_features, round(num_features * feature_multiplier**i))
                    for i in range(num_layers)]

        # Reverse order of image sizes and features for generator
        image_sizes = image_sizes[::-1]
        features = features[::-1]

        # Input layer
        if fully_convolutional:
            self.input_layer = G_block(num_latents, features[0], kernel_size=latent_kernel,
                                       stride=1, padding=0, **block_config)
        else:
            self.input_layer = nn.Sequential(
                nn.Flatten(),
                nn.Linear(num_latents, features[0] * image_sizes[0]**2, bias=False),
                View(features[0], image_sizes[0], image_sizes[0])
            )

        # Intermediate layers
        self.main_layers = nn.Sequential(*[
            G_block(in_features, out_features, kernel_size=4+(expected_size%2), **block_config)
            for in_features, out_features, expected_size in zip(features, features[1:], image_sizes[1:])
        ])

        # Output layer
        self.output_layer = nn.ConvTranspose2d(features[-1], image_channels, kernel_size=4+(image_size%2),
                                               stride=2, padding=1, bias=False)
        self.output_activation = nn.Tanh()

    def forward(self, x):
        # Add H and W dimensions, infer channels dim (add if none)
        x = x.view(x.size(0), -1, 1, 1)
        x = self.input_layer(x)
        x = self.main_layers(x)
        x = self.output_layer(x)
        x = self.output_activation(x)
        return x


class DCGAN(nn.Module):
    """Deep Convolutional Generative Adversarial Network"""

    def __init__(self,
                 num_latents=100,
                 num_features=64,
                 image_channels=3,
                 image_size=64,
                 feature_multiplier=2,
                 gan_type="gan",
                 fully_convolutional=True,
                 activation=None,
                 use_batchnorm=True,
                 use_spectralnorm=False,):
        """
        Initializes DCGAN.

        Args:
            num_latents: Number of latent factors.
            num_features: Number of features in the convolutions.
            image_channels: Number of channels in the input image.
            image_size: Size (i.e. height or width) of image.
            gan_type: Type of GAN (e.g. "gan" or "wgan-gp").
        """
        super().__init__()

        self.num_latents = num_latents
        self.num_features = num_features
        self.image_channels = image_channels
        self.image_size = image_size
        self.feature_multiplier = feature_multiplier
        self.gan_type = gan_type
        self.fully_convolutional = fully_convolutional
        self.activation = activation
        self.use_batchnorm = use_batchnorm
        self.use_spectralnorm = use_spectralnorm

        D_params = {
            "num_latents": 1,  # XXX
            "num_features": num_features,
            "image_channels": image_channels,
            "image_size": image_size,
            "feature_multiplier": feature_multiplier,
            "gan_type": gan_type,
            "fully_convolutional": fully_convolutional,
            "activation": activation,
            "use_batchnorm": use_batchnorm,
            "use_spectralnorm": use_spectralnorm,
        }
        G_params = {
            "num_latents": num_latents,
            "num_features": num_features,
            "image_channels": image_channels,
            "image_size": image_size,
            "feature_multiplier": feature_multiplier,
            "gan_type": gan_type,
            "fully_convolutional": fully_convolutional,
            "activation": activation,
            "use_batchnorm": use_batchnorm,
            "use_spectralnorm": use_spectralnorm,
        }

        self.D = DCGAN_Discriminator(**D_params)
        self.G = DCGAN_Generator(**G_params)


class View(nn.Module):
    def __init__(self, *shape, including_batch=False):
        super().__init__()
        self.shape = shape
        self.including_batch = including_batch
    
    def forward(self, x):
        if self.including_batch:
            return x.view(*self.shape)
        else:
            return x.view(x.size(0), *self.shape)

class ChannelNoise(nn.Module):
    """
    Channel noise injection module.
    Adds a linearly transformed noise to a convolution layer.
    """

    def __init__(self, num_channels, std=0.02):
        super().__init__()
        self.std = std
        self.scale = nn.Parameter(torch.ones(1, num_channels, 1, 1))


    def forward(self, x):
        noise_size = [x.size()[0], 1, *x.size()[2:]]  # single channel
        noise = self.std * torch.randn(noise_size).to(x)

        return x + self.scale * noise

### SimSiam

In [5]:
class SimSiam(nn.Module):
    """
    Build a SimSiam model.
    """
    def __init__(self, base_encoder, dim=2048, pred_dim=512):
        """
        dim: feature dimension (default: 2048)
        pred_dim: hidden dimension of the predictor (default: 512)
        """
        super(SimSiam, self).__init__()

        # create the encoder
        # num_classes is the output fc dimension, zero-initialize last BNs
        self.encoder = base_encoder(num_classes=dim, zero_init_residual=True)

        # build a 3-layer projector
        prev_dim = self.encoder.fc.weight.shape[1]
        self.encoder.fc = nn.Sequential(nn.Linear(prev_dim, prev_dim, bias=False),
                                        nn.BatchNorm1d(prev_dim),
                                        nn.ReLU(inplace=True), # first layer
                                        nn.Linear(prev_dim, prev_dim, bias=False),
                                        nn.BatchNorm1d(prev_dim),
                                        nn.ReLU(inplace=True), # second layer
                                        self.encoder.fc,
                                        nn.BatchNorm1d(dim, affine=False)) # output layer
        self.encoder.fc[6].bias.requires_grad = False # hack: not use bias as it is followed by BN

        # build a 2-layer predictor
        self.predictor = nn.Sequential(nn.Linear(dim, pred_dim, bias=False),
                                        nn.BatchNorm1d(pred_dim),
                                        nn.ReLU(inplace=True), # hidden layer
                                        nn.Linear(pred_dim, dim)) # output layer

    def forward(self, x1, x2):
        """
        Input:
            x1: first views of images
            x2: second views of images
        Output:
            p1, p2, z1, z2: predictors and targets of the network
            See Sec. 3 of https://arxiv.org/abs/2011.10566 for detailed notations
        """

        # compute features for one view
        z1 = self.encoder(x1) # NxC
        z2 = self.encoder(x2) # NxC

        p1 = self.predictor(z1) # NxC
        p2 = self.predictor(z2) # NxC

        return p1, p2, z1.detach(), z2.detach()

### ConsistentGAN

In [30]:
class ConsistentGAN(nn.Module):
    def __init__(self,
                 base_encoder,
                 repr_dim,
                 pred_dim,
                 latent_dim,
                 image_size=64,
                 *args, **kwargs):
        """
        s ~ S is representation/encoding space, e.g. s = Enc(x).
        z ~ Z is latent/seed space for G, e.g. x_fake ~ G(z).

        Case A:
            - Sample x_real ~ X
            - s <- Encoder(x_real)
            - Sample z ~ Z (coupled with s somehow? e.g. z = f(s) + noise)
            - x_fake <- G(z)
            - Test D on x_real, x_fake, i.e. test whether x in X}
            - Calculate GAN loss
            - Calculate SimSiam loss with Predictor(s)

        Case B: 
            - Sample x_real ~ X
            - s_real <- Encoder(x_real)
            - x_recon <- Decoder(s_real)
            - Sample z ~ Z (e.g. N(0,1) or U(-1,1))
            - s_fake <- G(z)
            - Test D on s_real, s_fake, i.e. test whether Dec(s) in X (how?)
            - Calculate GAN loss
            - Calculate Contrastive/SimSiam loss with Predictor(s)
            - Calculate reconsruction loss, e.g. || Enc(x_recon) - Enc(x_real) ||

        We choose Case B for now.

        Adversarial learning:
            Case A:
                Train D and G so that we can disciminate x based on its representation.
                Train G so that representation has enough info for reconstructing x.
                Ideally, G would generate different views of x given similar s's.
            Case B:
                Train D and G so that we can disciminate representations.
                Train G so that it produces representations as real (close to Enc(x)) as possible.
                This assumes that Enc(x) is the real representation.
                (There is no such thing as any random transformation can be real enough, but
                we should try to mitigate this problem nonetheless. For example, we can at least
                make it stable enough by pre-training Enc/Dec.)
                Ideally, G would produce accurate representations that can be decoded later.

        Contrastive learning:
            If x1 and x2 are views of same x (e.g. x1, x2 = rand_aug(x), rand_aug(x))
            Then, Encoder(x1) should be similar to Encoder(x2), so we do this (SimSiam algorithm):
            min 0.5*{ sim(Predictor(s1), s2.detach()) + sim(Predictor(s2), s1.detach()) }
            where sim = CosineSimilarity(dim=1).
        
        Reconstruction learning:
            We can add a (deterministic) Decoder that learns to decode s to x.
            This does not take into account the invariance of representation to augmentations.
            It simply learns the inverse of Encoder. G will learn to produce represenations
            that are as real as possible, where the Decoder will learn to decode them into their
            corresponding x's such that Enc(Dec(s)) = s (do we stop grad at s for this loss?)

        Architecture:
            We assume x comes from an image dataset, e.g. CIFAR10.
            Encoder-Decoder pair can have a DCGAN-style architecture.
            The generator can also have an arch similar to the decoder.
            Other networks can be simple FCNs.
        """
        super().__init__()
        self.repr_dim = repr_dim
        self.latent_dim = latent_dim
        self.pred_dim = pred_dim

        ### Copied from SimSiam repo >>>>>>>>
        # create the encoder
        # num_classes is the output fc dimension, zero-initialize last BNs
        self.encoder = base_encoder(num_classes=repr_dim, zero_init_residual=True)

        # build a 3-layer projector
        prev_dim = self.encoder.fc.weight.shape[1]
        self.encoder.fc = nn.Sequential(nn.Linear(prev_dim, prev_dim, bias=False),
                                        nn.BatchNorm1d(prev_dim),
                                        nn.ReLU(inplace=True), # first layer
                                        nn.Linear(prev_dim, prev_dim, bias=False),
                                        nn.BatchNorm1d(prev_dim),
                                        nn.ReLU(inplace=True), # second layer
                                        self.encoder.fc,
                                        nn.BatchNorm1d(repr_dim, affine=False)) # output layer
        self.encoder.fc[6].bias.requires_grad = False # hack: not use bias as it is followed by BN

        # build a 2-layer predictor
        self.predictor = nn.Sequential(nn.Linear(repr_dim, pred_dim, bias=False),
                                        nn.BatchNorm1d(pred_dim),
                                        nn.ReLU(inplace=True), # hidden layer
                                        nn.Linear(pred_dim, repr_dim)) # output layer
        ### <<<<<<<<
        
        # Make D's architecture kinda similar to predictor @TODO
        # @XXX: Shouldn't use batchnorm with grad_penalty
        D_hidden_dim = repr_dim // 2
        self.D = nn.Sequential(nn.Linear(repr_dim, D_hidden_dim, bias=False),
                               nn.BatchNorm1d(D_hidden_dim),
                               nn.LeakyReLU(0.2, inplace=True),
                               nn.Linear(D_hidden_dim, D_hidden_dim, bias=False),
                               nn.BatchNorm1d(D_hidden_dim),
                               nn.LeakyReLU(0.2, inplace=True),
                               nn.Linear(D_hidden_dim, D_hidden_dim, bias=False),
                               nn.BatchNorm1d(D_hidden_dim),
                               nn.LeakyReLU(0.2, inplace=True),
                               nn.Linear(D_hidden_dim, 1),
                               nn.Sigmoid())

        # Same for generator (latent -> representations)
        self.G = nn.Sequential(nn.Linear(latent_dim, repr_dim, bias=False),
                               nn.BatchNorm1d(repr_dim),
                               nn.ReLU(inplace=True), # hidden layer
                               nn.Linear(repr_dim, repr_dim, bias=False),
                               nn.BatchNorm1d(repr_dim),
                               nn.ReLU(inplace=True), # hidden layer
                               nn.Linear(repr_dim, repr_dim, bias=False),
                               nn.BatchNorm1d(repr_dim),
                               nn.ReLU(inplace=True), # hidden layer
                               nn.Linear(repr_dim, repr_dim))

        # Decoder should be a ConvT net. We'll use DCGAN's G for now @XXX
        self.decoder = DCGAN_Generator(num_latents=repr_dim, image_size=image_size)

        # To check progress of G
        self.fixed_latent = self.sample_latent(8*8)
    

    def sample_latent(self, batch_size):

        latent_size = [batch_size, self.latent_dim]
        latent = torch.randn(latent_size)

        return latent

    def forward(self, x1, x2):
        """
        Input:
            x1: first views of images
            x2: second views of images
        Output:
            p1, p2, z1, z2: predictors and targets of the network
            See Sec. 3 of https://arxiv.org/abs/2011.10566 for detailed notations
        """

        # compute features for one view
        z1 = self.encoder(x1) # NxC
        z2 = self.encoder(x2) # NxC

        p1 = self.predictor(z1) # NxC
        p2 = self.predictor(z2) # NxC

        return p1, p2, z1.detach(), z2.detach()


class ConsistentGAN2(nn.Module):
    def __init__(self, repr_dim, latent_dim,
                 D_batchnorm=True, image_size=64, *args, **kwargs):
        super().__init__()
        self.repr_dim = repr_dim
        self.latent_dim = latent_dim
        
        # Make D's architecture kinda similar to predictor
        D_hidden_dim = repr_dim // 10
        if D_batchnorm:
            self.D = nn.Sequential(nn.Linear(repr_dim, D_hidden_dim, bias=False),
                                   nn.LeakyReLU(0.2, inplace=True),
                                   nn.Linear(D_hidden_dim, D_hidden_dim, bias=False),
                                   nn.BatchNorm1d(D_hidden_dim),
                                   nn.LeakyReLU(0.2, inplace=True),
                                   nn.Linear(D_hidden_dim, 1),
                                   nn.Sigmoid())
        else:
            self.D = nn.Sequential(nn.Linear(repr_dim, D_hidden_dim, bias=False),
                                   nn.LeakyReLU(0.2, inplace=True),
                                   nn.Linear(D_hidden_dim, D_hidden_dim, bias=False),
                                   nn.LeakyReLU(0.2, inplace=True),
                                   nn.Linear(D_hidden_dim, 1),
                                   nn.Sigmoid())


        # Same for generator (latent -> representations)
        G_hidden_dim = repr_dim // 10
        self.G = nn.Sequential(nn.Linear(latent_dim, G_hidden_dim, bias=False),
                               nn.BatchNorm1d(G_hidden_dim),
                               nn.LeakyReLU(0.2, inplace=True),
                               #nn.Linear(G_hidden_dim, G_hidden_dim, bias=False),
                               #nn.BatchNorm1d(G_hidden_dim),
                               #nn.LeakyReLU(0.2, inplace=True),
                               #nn.Linear(G_hidden_dim, G_hidden_dim, bias=False),
                               #nn.BatchNorm1d(G_hidden_dim),
                               #nn.LeakyReLU(0.2, inplace=True),
                               nn.Linear(G_hidden_dim, G_hidden_dim, bias=False),
                               nn.BatchNorm1d(G_hidden_dim),
                               nn.LeakyReLU(0.2, inplace=True),
                               nn.Linear(G_hidden_dim, repr_dim),
                               )

        # Encodes x to context
        # XXX
        ctx_dim = repr_dim // 10
        self.ctx_encoder = DCGAN_Discriminator(num_latents=ctx_dim, image_size=image_size)
        # Projections to decoding space
        repr_dec_dim = repr_dim // 10
        ctx_dec_dim = ctx_dim // 10
        num_latents = repr_dec_dim + ctx_dec_dim
        self.repr_proj = nn.Linear(repr_dim, repr_dec_dim)
        self.ctx_proj = nn.Linear(ctx_dim, ctx_dec_dim)
        # Decodes representation + context projection to an image
        self.decoder = DCGAN_Generator(num_latents=num_latents, image_size=image_size)

        # To check progress of G
        self.fixed_latent = self.sample_latent(8*8)
    
    # XXX temp function
    def proj(self, repr, ctx):
        return torch.cat([self.repr_proj(repr), self.ctx_proj(ctx)], dim=1)

    def sample_latent(self, batch_size):
        latent_size = [batch_size, self.latent_dim]
        latent = torch.randn(latent_size)
        return latent


# Training v2

### Imports and globals

In [7]:
import argparse
import builtins
import math
import os
import random
import shutil
import time
import warnings
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

GANSIAM_DIR = "/content/drive/My Drive/gansiam/"
SIMSIAM_PATH = os.path.join(GANSIAM_DIR, "pretrained_batch256.tar")
TINYIMAGENET_DIR = "tiny-imagenet-200"
IMAGE_SIZE = 64
GENERATED_GRIDS = []

### Download Tiny Imagenet

In [8]:
%%bash
if [[ -d  "tiny-imagenet-200" ]]; then
    echo "Tiny Imagenet exists."
else
    wget -q "http://cs231n.stanford.edu/tiny-imagenet-200.zip"
    unzip -qq "tiny-imagenet-200.zip" && rm "tiny-imagenet-200.zip"
    echo "Downloaded Tiny Imagenet."
fi

Downloaded Tiny Imagenet.


### Utility

#### GAN Utils

In [9]:
def get_D_loss(gan_type="gan"):
    if gan_type in ("gan", "gan-gp"):
        return D_loss_GAN
    elif gan_type in ("wgan", "wgan-gp"):
        return D_loss_WGAN
    else:
        raise ValueError(f"gan_type {gan_type} not supported")


def get_G_loss(gan_type="gan"):
    if gan_type in ("gan", "gan-gp"):
        return G_loss_GAN
    elif gan_type in ("wgan", "wgan-gp"):
        return G_loss_WGAN
    else:
        raise ValueError(f"gan_type {gan_type} not supported")


def D_loss_GAN(D_real, D_fake, label_smoothing=True):
    
    # Create (noisy) real and fake labels XXX
    if label_smoothing:
        real_label = 0.7 + 0.5 * torch.rand_like(D_real)
    else:
        real_label = torch.ones_like(D_real) - 0.1
    fake_label = torch.zeros_like(D_fake)

    # Calculate binary cross entropy loss
    D_loss_real = F.binary_cross_entropy(D_real, real_label)
    D_loss_fake = F.binary_cross_entropy(D_fake, fake_label)

    # Loss is: - log(D(x)) - log(1 - D(x_g)),
    # which is equiv. to maximizing: log(D(x)) + log(1 - D(x_g))
    D_loss = D_loss_real + D_loss_fake

    return D_loss.mean()


def D_loss_WGAN(D_real, D_fake):

    # Maximize: D(x) - D(x_g) - const * (|| grad of D(x_i) wrt x_i || - 1)^2,
    # where x_i <- eps * x + (1 - eps) * x_g, and eps ~ rand(0,1)
    D_loss = -1 * (D_real - D_fake)

    return D_loss.mean()


def G_loss_GAN(D_fake):

    # Calculate binary cross entropy loss with a fake binary label
    fake_label = torch.zeros_like(D_fake)

    # Loss is: -log(D(G(z))), which is equiv. to minimizing log(1-D(G(z)))
    # We use this loss vs. the original one for stability only.
    G_loss = F.binary_cross_entropy(D_fake, 1 - fake_label)

    return G_loss.mean()


def G_loss_WGAN(D_fake):

    # Minimize: -D(G(z))
    G_loss = -D_fake
    
    return G_loss.mean()


def interpolate(real, fake):
    eps_size = [1] * len(real.size())
    eps_size[0] = real.size(0)
    eps = torch.rand(eps_size).to(real)
    return eps * real + (1 - eps) * fake

def simple_gradient_penalty(D, x, center=0.):
    x.requires_grad_()
    D_x = D(x)
    D_grad = torch.autograd.grad(D_x, x, torch.ones_like(D_x), create_graph=True)
    D_grad_norm = D_grad[0].view(x.size(0), -1).norm(dim=1)
    return (D_grad_norm - center).pow(2).mean()


#### Data Utils

In [10]:
from PIL import ImageFilter
import random


class TwoCropsTransform:
    """Take two random crops of one image as the query and key."""

    def __init__(self, base_transform):
        self.base_transform = base_transform

    def __call__(self, x):
        q = self.base_transform(x)
        k = self.base_transform(x)
        return [q, k]


class GaussianBlur(object):
    """Gaussian blur augmentation in SimCLR https://arxiv.org/abs/2002.05709"""

    def __init__(self, sigma=[.1, 2.]):
        self.sigma = sigma

    def __call__(self, x):
        sigma = random.uniform(self.sigma[0], self.sigma[1])
        x = x.filter(ImageFilter.GaussianBlur(radius=sigma))
        return x

#### Train Utils

In [11]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    filename = os.path.join(GANSIAM_DIR, "results", filename)
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')


def adjust_learning_rate(optimizer, init_lr, epoch, args):
    """Decay the learning rate based on schedule"""
    cur_lr = init_lr * 0.5 * (1. + math.cos(math.pi * epoch / args.epochs))
    for param_group in optimizer.param_groups:
        if 'fix_lr' in param_group and param_group['fix_lr']:
            param_group['lr'] = init_lr
        else:
            param_group['lr'] = cur_lr


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


### Load pre-trained SimSiam model


In [31]:
checkpoint = torch.load(SIMSIAM_PATH, map_location="cuda:0")
# remove 'module.' from dict keys
model_dict = OrderedDict((k[7:], v) for k, v in checkpoint["state_dict"].items())

# Load model
simsiam = SimSiam(models.__dict__["resnet50"])
simsiam.load_state_dict(model_dict)
#print(simsiam)

<All keys matched successfully>

### Declare arguments

In [46]:
class Args:
    def __init__(self):
        self.data = TINYIMAGENET_DIR
        self.workers = 2
        self.epochs = 200
        self.batch_size = 256
        self.D_lr = 1e-4
        self.G_lr = 2e-4
        self.decoder_lr = 1e-5
        self.momentum = 0.9
        self.weight_decay = 1e-4
        self.print_freq = 10
        self.seed = None
        self.gpu = 0

        # SimSiam
        self.dim = 2048
        self.pred_dim = 512

        # GAN
        self.gan_type = "wgan-gp"
        self.repr_dim = self.dim  # don't change
        self.latent_dim = 256
        self.D_iters = 3
        self.noise = 1e-6
        self.grad_penalty = 10.
        self.grad_center = 1.
        self.generate_grid_interval = 50

        self.norm_penalty = 10.

GENERATED_GRIDS = []
args = Args()

### Init model and optimizers

In [47]:
if args.seed is not None:
    random.seed(args.seed)
    torch.manual_seed(args.seed)
    cudnn.deterministic = True

torch.cuda.set_device(args.gpu)

model = ConsistentGAN2(args.repr_dim, args.latent_dim,
                       D_batchnorm=args.gan_type in ("gan", "wgan"),
                       image_size=IMAGE_SIZE)
model = model.cuda(args.gpu)
simsiam = simsiam.cuda(args.gpu)
#print(model)

# Define D and G loss functions
D_criterion = get_D_loss(args.gan_type)
G_criterion = get_G_loss(args.gan_type)
def rec_criterion(x, y):
    return (x - y).pow(2).flatten(start_dim=1).sum(dim=1).mean()

D_optimizer = torch.optim.SGD(
    model.D.parameters(), args.D_lr, momentum=args.momentum)
G_optimizer = torch.optim.SGD(
    model.G.parameters(), args.G_lr, momentum=args.momentum)
decoder_optimizer = torch.optim.SGD(
    model.decoder.parameters(), args.decoder_lr,
    momentum=args.momentum, weight_decay=args.weight_decay)

ctx_encoder_optimizer = torch.optim.SGD(
    model.ctx_encoder.parameters(), args.decoder_lr,
    momentum=args.momentum, weight_decay=args.weight_decay)

cudnn.benchmark = True

print("Num of params:", sum(map(torch.numel, model.parameters())))

Num of params: 10417389


### Data loading and transformation

In [48]:
# image normalization
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
normalize = transforms.Normalize(mean=mean, std=std)
inv_normalize = transforms.Normalize(
   mean= [-m/s for m, s in zip(mean, std)],
   std= [1/s for s in std]
)


# MoCo v2's aug: similar to SimCLR https://arxiv.org/abs/2002.05709
"""
augmentation = [
    transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.2, 1.)),
    transforms.RandomApply([
        transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)  # not strengthened
    ], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.RandomApply([GaussianBlur([.1, 2.])], p=0.5),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
]
"""


augmentation = [
    #transforms.RandomResizedCrop(IMAGE_SIZE),
    transforms.Resize(IMAGE_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
]



# Using CIFAR10
train_dataset = datasets.CIFAR10(
    root=os.path.join(GANSIAM_DIR, "cifar10/train"), train=True, download=True,
    transform=transforms.Compose(augmentation))
    #transform=TwoCropsTransform(transforms.Compose(augmentation)))
"""

# Using Tiny Imagenet
train_dataset = datasets.ImageFolder(
    root=os.path.join(TINYIMAGENET_DIR, 'train'),
    transform=transforms.Compose(augmentation))
"""

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=args.batch_size, shuffle=True,
    num_workers=args.workers, pin_memory=True, sampler=None, drop_last=True)

Files already downloaded and verified


### Training

In [49]:
def train(train_loader, model, simsiam,
          rec_criterion, D_criterion, G_criterion,
          D_optimizer, G_optimizer,
          decoder_optimizer, ctx_encoder_optimizer,
          epoch, args):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    D_on_reals = AverageMeter('D(real)', ':.4f')
    D_on_fakes = AverageMeter('D(fake)', ':.4f')
    D_grads = AverageMeter('grad(D) penalty', ':.4f')
    rec_losses = AverageMeter('Rec loss', ':.4f')
    norms = AverageMeter('norm', ':.4f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time,
         D_on_reals, D_on_fakes, D_grads, rec_losses, norms],
        prefix="Epoch: [{}]".format(epoch))

    # switch to train mode
    model.train()

    end = time.time()
    for i, (x, y) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        x = x.cuda(args.gpu, non_blocking=True)
        batch_size = x.size(0)

        # compute output and loss
        with torch.no_grad():
            repr = simsiam.encoder(x)

        ### train GAN
        if (i+1) % (args.D_iters+1) > 0:
            latent = model.sample_latent(batch_size).cuda(args.gpu)
            
            # Add noise to real sample
            real = repr + args.noise * torch.randn_like(repr)

            # Sample from generator
            with torch.no_grad():
                fake = model.G(latent)
                # Add noise to fake sample as well
                fake += torch.randn_like(fake) * args.noise

            # Classify real and fake data
            D_real = model.D(real)
            D_fake = model.D(fake)

            # Calculate loss
            D_loss = D_criterion(D_real, D_fake)
            # Gradient penalty
            D_grad_penalty = simple_gradient_penalty(
                model.D, interpolate(real, fake), center=args.grad_center)
            D_loss += args.grad_penalty * D_grad_penalty

            # Calculate gradient and minimize
            D_optimizer.zero_grad()
            D_loss.backward()
            D_optimizer.step()

        else:
            # Sample from generators
            latent = model.sample_latent(batch_size).cuda(args.gpu)
            fake = model.G(latent)
            fake += args.noise * torch.randn_like(fake)
            # Classify fake images
            D_fake = model.D(fake)
            # Calculate loss
            G_loss = G_criterion(D_fake)
            # Calculate gradient and minimize
            G_optimizer.zero_grad()
            G_loss.backward()
            G_optimizer.step()

            # Save D's evaluations on real and fake
            D_on_reals.update(D_real.mean().item(), batch_size)
            D_on_fakes.update(D_fake.mean().item(), batch_size)
            D_grads.update(D_grad_penalty.mean().item(), batch_size)

        # Calculate Reconstruction loss
        ctx = model.ctx_encoder(x)
        proj = model.proj(repr, ctx)
        x_rec = model.decoder(proj + 1e-5 * torch.randn_like(proj))
        # loss
        rec_loss = rec_criterion(x, x_rec)
        proj_norm = proj.flatten(start_dim=1).norm(dim=1).mean()
        
        decoder_optimizer.zero_grad()
        ctx_encoder_optimizer.zero_grad()
        (rec_loss + args.norm_penalty * proj_norm).backward()
        decoder_optimizer.step()
        ctx_encoder_optimizer.step()

        rec_losses.update(rec_loss.item(), batch_size)
        norms.update(proj_norm.item(), batch_size)

        # Check generator's progress by recording its output on a fixed input
        if i % args.generate_grid_interval == 0:
            def decode_latent(z):
                repr = model.G(z)
                ctx = model.ctx_encoder(x[:model.fixed_latent.size(0)])  # ref x
                proj = model.proj(repr, ctx)
                return model.decoder(proj)

            grid = generate_grid(decode_latent, model.fixed_latent.cuda(args.gpu))
            GENERATED_GRIDS.append(grid)

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            progress.display(i)


In [50]:
for epoch in range(args.epochs):
    # train for one epoch
    train(train_loader, model, simsiam,
          rec_criterion, D_criterion, G_criterion,
          D_optimizer, G_optimizer, 
          decoder_optimizer, ctx_encoder_optimizer,
          epoch, args)

    if epoch % 10 == 0:
        save_checkpoint({
            'state_dict': model.state_dict(),
        }, is_best=False, filename='checkpoint_{:04d}.pth.tar'.format(epoch))

/usr/local/lib/python3.7/dist-packages/torchvision/utils.py:50: UserWarning: range will be deprecated, please use value_range instead.
  warnings.warn(warning)


Epoch: [0][  0/195]	Time  0.394 ( 0.394)	Data  0.207 ( 0.207)	D(real) 0.0000 (0.0000)	D(fake) 0.0000 (0.0000)	grad(D) penalty 0.0000 (0.0000)	Rec loss 19063.2539 (19063.2539)	norm 1.2087 (1.2087)
Epoch: [0][ 10/195]	Time  0.151 ( 0.174)	Data  0.000 ( 0.019)	D(real) 0.4852 (0.4852)	D(fake) 0.4901 (0.4903)	grad(D) penalty 0.9476 (0.9478)	Rec loss 12014.6602 (14027.4885)	norm 1.6422 (1.3665)
Epoch: [0][ 20/195]	Time  0.147 ( 0.161)	Data  0.000 ( 0.010)	D(real) 0.4853 (0.4852)	D(fake) 0.4904 (0.4903)	grad(D) penalty 0.9466 (0.9475)	Rec loss 9657.6230 (12401.6411)	norm 1.9530 (1.5992)
Epoch: [0][ 30/195]	Time  0.148 ( 0.157)	Data  0.000 ( 0.007)	D(real) 0.4853 (0.4853)	D(fake) 0.4891 (0.4900)	grad(D) penalty 0.9457 (0.9471)	Rec loss 8894.6113 (11546.1828)	norm 2.2423 (1.7648)
Epoch: [0][ 40/195]	Time  0.150 ( 0.156)	Data  0.000 ( 0.005)	D(real) 0.4853 (0.4853)	D(fake) 0.4887 (0.4897)	grad(D) penalty 0.9439 (0.9463)	Rec loss 8633.0205 (10891.4995)	norm 2.2369 (1.8677)
Epoch: [0][ 50/195]	Tim

### Check results

In [51]:
vidname = os.path.join(GANSIAM_DIR, "results/test.mp4")
create_progress_animation(GENERATED_GRIDS, vidname)

In [45]:
%matplotlib inline
def show(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')

def show_sample():
    model.eval()
    with torch.no_grad():
        z = model.sample_latent(16).cuda(args.gpu)
        im_tensor = model.decoder(model.G(z))
        im_tensor = inv_normalize(im_tensor)
        im_grid = vutils.make_grid(im_tensor.cpu(), padding=2, nrow=4)
        plt.imshow(im_grid.permute(1,2,0))

show_sample()

RuntimeError: ignored

In [ ]:
# Recreate a batch of images from the dataset
for x, y in train_loader:
    with torch.no_grad():
        #x_rec = model.decoder(simsiam.encoder(x.cuda(args.gpu)))
        x = x.cuda(args.gpu)
        x_rec, _ = model.decoder_with_proj(simsiam.encoder(x), model.encoder(x))
    break

# Actual
plt.figure()
plt.axis('off')
plt.imshow(vutils.make_grid(
    inv_normalize(x)[:16].cpu(), padding=2, nrow=4).permute(1,2,0))
# Reconstruction
plt.figure()
plt.axis('off')
plt.imshow(vutils.make_grid(
    inv_normalize(x_rec)[:16].cpu(), padding=2, nrow=4).permute(1,2,0))